# Building Database for Crime Reports

## Introduction
In this project, we will be creating a database for storing data related to crimes that occurred in Boston. We will also practice creating the table inside a schema, making sure that all the datatypes are correct. Finally, we will also create different user groups which will have different privileges whenever they access the database on our server.

## Creating the Crime Database

In order to create the database, we will have to connect to our Postgres server first. After creating the database, we will also create the schema where our table will be located.

In [1]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

user = config['postgresql']['user']
password = config['postgresql']['password']

In [2]:
import psycopg2

conn = psycopg2.connect(user=user, password=password)
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.autocommit = False
conn.close()

In [3]:
# Create schema
conn = psycopg2.connect(dbname='crime_db', user=user, password=password)
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")

## Obtaining Column Names and Sample

Here we will open the file containing the dataset and inspect the header row as well as the first data row to make sure we use the correct datatypes in our table.

In [4]:
import csv

with open('boston.csv', 'r', encoding='latin-1') as f:
    reader = csv.reader(f)
    data = list(reader)
    col_headers = data[0]
    first_row = data[1]
    print(col_headers)
    print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating an Auxiliary Function

Now we will create a function called `get_col_set()` which will return a set with all distinct values contained in a column given the name of a CSV file and a column index. We will use this function to identify the correct datatypes for our columns.

In [5]:
def get_col_set(csv_filename, col_index, encoding=None):
    with open(csv_filename, 'r', encoding=encoding) as f:
        next(f)  # Skips the header
        reader = csv.reader(f)

        distinct_values = set()
        for row in reader:
            distinct_values.add(row[col_index])

    return distinct_values

In [6]:
for index in range(len(col_headers)):
    distinct_values = get_col_set('boston.csv', index, encoding='latin-1')
    print(col_headers[index], len(distinct_values))

incident_number 298329
offense_code 219
description 239
date 1177
day_of_the_week 7
lat 18177
long 18177


## Finding Maximum Length of Values

In order to pick our datatypes correctly, it would be useful to calculate the longest length of any given value for each column. This way, we can make sure to keep the datatypes as simple as possible. Since the `description` column is likely to have the longest values, we will inspect that column first.

In [7]:
description_values = get_col_set('boston.csv', 2, encoding='latin-1')
max_len = 0
for value in description_values:
    max_len = max(max_len, len(value))
print(max_len)

58


The maximum length of any given value in the `description` column is 58. To be on the safe side, we will set `n` to be 100. This will give us a varchar datatype in which any string value cannot exceed more than 100 characters.

## Creating the Table

Before creating the table, we can see that the `day_of_the_week` column only contains 7 values. Since each value is unique in this set, we will create an `ENUM` datatype for this column. This is the only datatype we will explicitly create, and we will have to do this before creating the table.

In [8]:
cur.execute("""
    CREATE TYPE DAY_ENUM AS ENUM (
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")

In [9]:
# Create table
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_id INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_week DAY_ENUM,
        lat DECIMAL,
        long DECIMAL  
    );
""")

## Loading the Data

Now we will load the data from the `boston.csv` file into the new `crimes.boston_crimes` table that we just made. To do this, we will be using the `cursor.copy_expert()` method.

In [10]:
with open('boston.csv', 'r', encoding='latin-1') as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

# Check if data was loaded successfully
cur.execute("SELECT * FROM crimes.boston_crimes;")
data = cur.fetchall()
# Print first row, number of rows
print(data[0])
print(len(data))

(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', Decimal('42.35779134'), Decimal('-71.13937053'))
298329


## Revoking Public Privileges

Now that our data has loaded into our table successfully, we need to make sure that anyone who tries to access it publicly is denied of that access. To do this, we need to revoke all public privileges from the database and schema. By following the least privilege principle, we can revoke all privileges and then give the user or groups minimum access, if necessary.

In [11]:
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")

## Creating User Groups

Now that we have revoked public access, we want to create a `readonly` group which will only have access to use the `SELECT` statement. We also want to create another group called `readwrite` which will only have access to `SELECT`, `INSERT`, `DELETE`, and `UPDATE` statements.

In [12]:
# Create readonly and readwrite groups
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite NOLOGIN;")

# Granting access to database and schema
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

# Granting specific group privileges
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("""
    GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;
""")

## Creating Users

Now we will practice creating one user for each group.

In [13]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

# Commit all changes to the connection and close it
conn.commit()
conn.close()

## Testing Database

Now that we have finished creating our Postgres database, we will test to see if everything is configured as expected. We will query Postgres internal tables to do so.

In [14]:
# Testing for specific roles
conn2 = psycopg2.connect(dbname='crime_db', user=user, password=password)
cur2 = conn2.cursor()

cur2.execute(""" 
    SELECT
        rolname,
        rolsuper,
        rolcreaterole,
        rolcreatedb,
        rolcanlogin
    FROM pg_roles
    WHERE rolname IN ('postgres', 'readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
cur2.fetchall()

[('postgres', True, True, True, True),
 ('readonly', False, False, False, False),
 ('readwrite', False, False, False, False),
 ('data_analyst', False, False, False, True),
 ('data_scientist', False, False, False, True)]

In [15]:
# Testing for table privileges
cur2.execute("""
    SELECT 
        grantor,
        grantee,
        table_catalog,
        table_schema,
        table_name,
        privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
cur2.fetchall()

[('postgres', 'readonly', 'crime_db', 'crimes', 'boston_crimes', 'SELECT'),
 ('postgres', 'readwrite', 'crime_db', 'crimes', 'boston_crimes', 'INSERT'),
 ('postgres', 'readwrite', 'crime_db', 'crimes', 'boston_crimes', 'SELECT'),
 ('postgres', 'readwrite', 'crime_db', 'crimes', 'boston_crimes', 'UPDATE'),
 ('postgres', 'readwrite', 'crime_db', 'crimes', 'boston_crimes', 'DELETE')]

In [16]:
# Close the connection
conn2.close()

After querying the Postgres internal tables and testing for user roles and privileges, we can confirm that all of our configurations are correct. Only the assigned users can log into the database. Additionally, the `readonly` group can only use the `SELECT` statement, while the `readwrite` group has more privileges.

## Conclusion

In this project we practiced creating a database as well as a table and schema to load real data from a CSV file. We also checked to make sure we were using the correct datatypes for our table. Finally, we practiced creating users and groups while also granting them specific roles and privileges. This last step is very important for maintaining data integrity and security.